In [1]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm

In [2]:
# SIR model parameters
contact_rate = 0.3
recovery_rate = 0.1
total_pop = 1000

# Initial states
inf_init = 12
suscept_init = total_pop - inf_init
rec_init = 0

# Observations
observations = [12, 157, 982, 532, 89]

# Number of particles
n_particles = 100

# Updating particles
def update_particles(particles, contact_rate, recovery_rate, total_pop):
    suscept, infect, recovered = particles[:, 0], particles[:, 1], particles[:, 2]
    new_infections = np.random.binomial(suscept.astype(int), 1.0 - np.exp(-contact_rate * infect / total_pop))
    new_recoveries = np.random.binomial(infect.astype(int), 1.0 - np.exp(-recovery_rate))
    suscept -= new_infections
    infect += new_infections - new_recoveries
    recovered += new_recoveries
    particles[:, 0], particles[:, 1], particles[:, 2] = suscept, infect, recovered
    return particles

# Calculation of importance weights
target_sd = 50.0
def get_importance(p_vals, mean):
    zero_trunc_vals = -p_vals / target_sd
    target = np.array([mean] * n_particles)
    return truncnorm.pdf(target, zero_trunc_vals, np.inf, loc=p_vals, scale=target_sd)

In [3]:
# Initialise particles
particles = np.zeros([n_particles, 3, len(observations)])
particles[:, 0, 0] = suscept_init
particles[:, 1, 0] = inf_init
particles[:, 2, 0] = rec_init

# Main loop
for o, obs in enumerate(observations[1:]):
    
    # Prediction
    new_particles = update_particles(particles[:, :, o], contact_rate, recovery_rate, total_pop)

    # Importance
    weights = get_importance(new_particles[:, 1], obs)
    norm_weights = weights / sum(weights)

    # Resampling
    indices = np.random.choice(range(n_particles), size=n_particles, p=norm_weights)
    new_particles = new_particles[indices]
    
    # Results   
    print(f"Observation: {obs}")
    print(f"Estimated susceptible: {np.mean(new_particles[:, 0]):.2f}")
    print(f"Estimated infectious: {np.mean(new_particles[:, 1]):.2f}")
    print(f"Estimated recovered: {np.mean(new_particles[:, 2]):.2f}\n")

    # Update
    particles[:, :, o + 1] = new_particles

Observation: 157
Estimated susceptible: 984.44
Estimated infectious: 14.43
Estimated recovered: 1.13

Observation: 982
Estimated susceptible: 975.91
Estimated infectious: 22.27
Estimated recovered: 1.82

Observation: 532
Estimated susceptible: 965.69
Estimated infectious: 30.02
Estimated recovered: 4.29

Observation: 89
Estimated susceptible: 956.69
Estimated infectious: 36.61
Estimated recovered: 6.70

